In [ ]:
# ============================================
# Notebook genérico de limpieza de datos
# ============================================

# Librerías necesarias
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

# 1. Cargar CSV
# Cambia 'archivo.csv' por tu dataset
df = pd.read_csv("listings.csv", encoding="utf-8", sep=",")
df

In [ ]:

# 2. Exploración inicial
print("Dimensiones del dataset:", df.shape)
display(df.head())
df.info()
display(df.describe(include="all"))


In [ ]:

# ============================================
# 3. Problemas a revisar
# ============================================

# --- Valores ausentes ---
print("\nValores nulos por columna:")
print(df.isnull().sum())




In [ ]:
# df.query("State.isnull()")

In [ ]:
# --- Duplicados ---
print("\nNúmero de filas duplicadas:", df.duplicated().sum())



In [ ]:
# df[df.duplicated(keep=False)]
# df

duplicados = df[df.duplicated(keep=False)]

# Ordenar por las columnas relevantes
duplicados_ordenados = duplicados.sort_values(by=df.columns.tolist())
duplicados_ordenados


In [ ]:
# Marcar duplicados
df['is_duplicate'] = df.duplicated(keep=False)

# Filtrar duplicados
duplicados_df = df[df['is_duplicate']].copy()

# Ordenar por todas las columnas del dataset (excluir la columna is_duplicate)
cols_to_sort = [col for col in df.columns if col != 'is_duplicate']
duplicados_ordenados = duplicados_df.sort_values(by=cols_to_sort)

# Resumen agrupado - agrupar por todas las columnas originales para contar duplicados idénticos
resumen_duplicados = (
    duplicados_df[cols_to_sort]
    .value_counts()
    .reset_index(name='count')
    .sort_values(by='count', ascending=False)
)

print(f"Total de filas duplicadas: {len(duplicados_df)}")
print(f"\nPrimeros duplicados ordenados (mostrando {min(10, len(duplicados_ordenados))} filas):")
display(duplicados_ordenados.head(10))

print(f"\nResumen de duplicados (grupos únicos con sus conteos):")
display(resumen_duplicados.head(20))

# Limpiar columna temporal
df.drop(columns=['is_duplicate'], inplace=True)


In [ ]:
# ============================================
# Visualización y análisis de duplicados
# ============================================

# Solo ejecutar si hay duplicados
if df.duplicated().sum() > 0:
    # Marcar duplicados nuevamente (temporal)
    df_temp = df.copy()
    df_temp['is_duplicate'] = df_temp.duplicated(keep=False)
    duplicados_temp = df_temp[df_temp['is_duplicate']].copy()
    
    # 1. Gráfico de barras mejorado: Top 10 combinaciones más duplicadas
    cols_to_group = [col for col in df.columns]
    top_duplicados = (
        duplicados_temp[cols_to_group]
        .value_counts()
        .head(10)
        .reset_index(name='frecuencia')
    )
    
    # Crear figura más grande y con mejor distribución
    fig, ax = plt.subplots(figsize=(14, 8))
    
    # Crear etiquetas más descriptivas y legibles
    labels_completas = []
    labels_cortas = []
    for idx, row in top_duplicados.iterrows():
        # Identificar las columnas más relevantes (con valores variados)
        label_parts = []
        # Limitar a las primeras 4 columnas más relevantes
        for i, col in enumerate(cols_to_group[:4]):
            val = str(row[col])
            # Truncar valores muy largos
            if len(val) > 25:
                val = val[:22] + "..."
            label_parts.append(f"  • {col}: {val}")
        
        # Etiqueta completa para mostrar debajo del gráfico
        labels_completas.append(f"Grupo {idx+1}:\n" + "\n".join(label_parts))
        # Etiqueta corta para el eje X
        labels_cortas.append(f"Grupo {idx+1}")
    
    # Crear gráfico de barras con colores degradados
    colores = plt.cm.RdYlGn_r(np.linspace(0.3, 0.9, len(top_duplicados)))
    barras = ax.bar(range(len(top_duplicados)), top_duplicados['frecuencia'], 
                    color=colores, edgecolor='black', linewidth=1.5)
    
    # Agregar valores en la parte superior de cada barra
    for i, (barra, valor) in enumerate(zip(barras, top_duplicados['frecuencia'])):
        height = barra.get_height()
        ax.text(barra.get_x() + barra.get_width()/2., height,
                f'{int(valor)} registros\nduplicados',
                ha='center', va='bottom', fontsize=10, fontweight='bold',
                bbox=dict(boxstyle='round,pad=0.5', facecolor='yellow', alpha=0.7))
    
    # Configurar ejes y título
    ax.set_xlabel('Grupos de duplicados', fontsize=12, fontweight='bold')
    ax.set_ylabel('Número de registros duplicados', fontsize=12, fontweight='bold')
    ax.set_title('Top 10: Registros duplicados más frecuentes\n(Cuántas veces se repite exactamente la misma información)', 
                 fontsize=14, fontweight='bold', pad=20)
    ax.set_xticks(range(len(top_duplicados)))
    ax.set_xticklabels(labels_cortas, rotation=0, fontsize=11)
    
    # Agregar grid para mejor lectura
    ax.yaxis.grid(True, linestyle='--', alpha=0.7)
    ax.set_axisbelow(True)
    
    # Ajustar límites del eje Y
    ax.set_ylim(0, max(top_duplicados['frecuencia']) * 1.15)
    
    plt.tight_layout()
    plt.show()
    
    # Mostrar leyenda detallada de cada grupo
    print("\n" + "="*80)
    print("LEYENDA: Detalles de cada grupo de duplicados")
    print("="*80)
    for i, label in enumerate(labels_completas):
        print(f"\n{label}")
        print(f"  → Total de registros idénticos: {int(top_duplicados.iloc[i]['frecuencia'])}")
        print("-" * 80)
    
    # 2. Tabla cruzada: Analizar duplicados por columnas categóricas
    # Identificar columnas categóricas con menos de 20 valores únicos
    cat_cols = [col for col in df.select_dtypes(include=['object']).columns 
                if df[col].nunique() < 20 and df[col].nunique() > 1]
    
    if len(cat_cols) >= 2:
        # Crear tabla cruzada con las dos primeras columnas categóricas
        col1, col2 = cat_cols[0], cat_cols[1]
        
        # Marcar si cada fila es duplicada
        df_temp['es_duplicado'] = df_temp.duplicated(keep=False).astype(int)
        
        # Tabla cruzada: contar duplicados por categorías
        tabla_cruzada = pd.crosstab(
            df_temp[col1], 
            df_temp[col2],
            values=df_temp['es_duplicado'],
            aggfunc='sum',
            margins=True,
            margins_name='TOTAL'
        )
        
        print(f"\n{'='*80}")
        print(f"TABLA CRUZADA: Duplicados según '{col1}' vs '{col2}'")
        print(f"{'='*80}")
        print(f"(Muestra cuántos registros duplicados hay para cada combinación)\n")
        display(tabla_cruzada)
        
        # Heatmap de la tabla cruzada (sin la fila/columna de totales)
        if len(tabla_cruzada) > 2:  # Solo si hay suficientes datos
            plt.figure(figsize=(14, 10))
            sns.heatmap(
                tabla_cruzada.iloc[:-1, :-1],  # Excluir fila y columna 'TOTAL'
                annot=True, 
                fmt='g', 
                cmap='YlOrRd',
                cbar_kws={'label': 'Cantidad de duplicados'},
                linewidths=0.5,
                linecolor='gray'
            )
            plt.title(f'Mapa de Calor: Concentración de duplicados\nCruce entre {col1} y {col2}\n(Colores más oscuros = más duplicados)', 
                     fontsize=14, fontweight='bold', pad=20)
            plt.xlabel(col2, fontsize=12, fontweight='bold')
            plt.ylabel(col1, fontsize=12, fontweight='bold')
            plt.tight_layout()
            plt.show()
    
    # 3. Análisis detallado del primer grupo duplicado
    print("\n" + "="*80)
    print("EJEMPLO DETALLADO: Primer grupo de duplicados encontrado")
    print("="*80)
    print("(Mostrando todas las filas que tienen exactamente la misma información)\n")
    
    # Obtener el primer grupo de duplicados
    primer_grupo = (
        duplicados_temp[cols_to_group]
        .value_counts()
        .head(1)
        .reset_index(name='count')
    )
    
    if len(primer_grupo) > 0:
        # Crear filtro para encontrar todas las filas de este grupo
        filtros = []
        for col in cols_to_group:
            valor = primer_grupo.iloc[0][col]
            if pd.isna(valor):
                filtros.append(df_temp[col].isna())
            else:
                filtros.append(df_temp[col] == valor)
        
        # Combinar todos los filtros
        filtro_final = filtros[0]
        for f in filtros[1:]:
            filtro_final = filtro_final & f
        
        ejemplo_duplicados = df_temp[filtro_final].drop(columns=['is_duplicate', 'es_duplicado'], errors='ignore')
        
        print(f"✓ Este grupo tiene {len(ejemplo_duplicados)} registros idénticos\n")
        print("Registros duplicados:")
        display(ejemplo_duplicados)
        
        print(f"\n{'='*80}")
        print("Valores compartidos por todos estos registros duplicados:")
        print(f"{'='*80}")
        for i, col in enumerate(cols_to_group[:15], 1):  # Mostrar primeras 15 columnas
            valor = primer_grupo.iloc[0][col]
            print(f"{i:2}. {col:30} = {valor}")
        
        if len(cols_to_group) > 15:
            print(f"\n... y {len(cols_to_group) - 15} columnas más con valores idénticos")
    
    print("\n" + "="*80)
    
    # Resumen final
    total_duplicados = len(duplicados_temp)
    total_registros = len(df_temp)
    porcentaje = (total_duplicados / total_registros) * 100
    
    print("\n" + "="*80)
    print("RESUMEN DE DUPLICADOS")
    print("="*80)
    print(f"• Total de registros en el dataset: {total_registros:,}")
    print(f"• Total de registros duplicados: {total_duplicados:,}")
    print(f"• Porcentaje de duplicados: {porcentaje:.2f}%")
    print(f"• Grupos únicos de duplicados: {len(top_duplicados)}")
    print("="*80)
    
else:
    print("✓ No se encontraron duplicados en el dataset")


In [ ]:
# ============================================
# --- Inconsistencias de formato ---
# ============================================

import re
from collections import Counter

print("\n" + "="*80)
print("ANÁLISIS DE INCONSISTENCIAS DE FORMATO")
print("="*80)

# Función para detectar el formato de una fecha
def detectar_formato_fecha(valor):
    """Detecta el formato de una fecha"""
    if pd.isnull(valor):
        return 'NULL'
    
    valor_str = str(valor).strip()
    
    # Patrones comunes de fecha
    patrones = {
        'DD/MM/YY': r'^\d{1,2}/\d{1,2}/\d{2}$',
        'DD/MM/YYYY': r'^\d{1,2}/\d{1,2}/\d{4}$',
        'DD-MM-YY': r'^\d{1,2}-\d{1,2}-\d{2}$',
        'DD-MM-YYYY': r'^\d{1,2}-\d{1,2}-\d{4}$',
        'YYYY-MM-DD': r'^\d{4}-\d{1,2}-\d{1,2}$',
        'YYYY/MM/DD': r'^\d{4}/\d{1,2}/\d{1,2}$',
        'MM/DD/YYYY': r'^\d{1,2}/\d{1,2}/\d{4}$',  # Ambiguo con DD/MM/YYYY
        'YYYYMMDD': r'^\d{8}$',
        'YYYY-MM-DD HH:MM:SS': r'^\d{4}-\d{2}-\d{2}\s+\d{1,2}:\d{2}:\d{2}',
        'DD/MM/YYYY HH:MM': r'^\d{1,2}/\d{1,2}/\d{4}\s+\d{1,2}:\d{2}',
        'ISO8601': r'^\d{4}-\d{2}-\d{2}T\d{2}:\d{2}:\d{2}',
    }
    
    # Valores especiales
    if valor_str == '-' or valor_str == '':
        return 'PLACEHOLDER'
    
    # Buscar coincidencia con patrones
    for nombre, patron in patrones.items():
        if re.match(patron, valor_str):
            return nombre
    
    # Si contiene números y separadores, es formato desconocido
    if re.search(r'\d', valor_str) and re.search(r'[/-:]', valor_str):
        return 'FORMATO_DESCONOCIDO'
    
    return 'NO_FECHA'

# Función para detectar formato de hora
def detectar_formato_hora(valor):
    """Detecta el formato de una hora"""
    if pd.isnull(valor):
        return 'NULL'
    
    valor_str = str(valor).strip()
    
    if valor_str == '-' or valor_str == '':
        return 'PLACEHOLDER'
    
    patrones_hora = {
        'HH:MM:SS': r'^\d{1,2}:\d{2}:\d{2}$',
        'HH:MM': r'^\d{1,2}:\d{2}$',
        'HHMM': r'^\d{4}$',
        'HH:MM AM/PM': r'^\d{1,2}:\d{2}\s*(AM|PM|am|pm)$',
    }
    
    for nombre, patron in patrones_hora.items():
        if re.match(patron, valor_str):
            return nombre
    
    return 'NO_HORA'

# Función genérica para detectar formato de cualquier campo
def detectar_formato_generico(valor):
    """Detecta el patrón general de un valor"""
    if pd.isnull(valor):
        return 'NULL'
    
    valor_str = str(valor).strip()
    
    if valor_str == '-' or valor_str == '':
        return 'PLACEHOLDER'
    
    # Crear patrón simplificado
    patron = re.sub(r'\d', 'D', valor_str)  # Dígitos -> D
    patron = re.sub(r'[a-zA-Z]', 'A', patron)  # Letras -> A
    patron = re.sub(r'\s+', ' ', patron)  # Normalizar espacios
    
    return patron[:50]  # Limitar longitud

# Analizar todas las columnas de texto
problemas_formato = {}

for col in df.select_dtypes(include="object").columns:
    # Detectar tipo de columna
    nombre_lower = col.lower()
    
    # Determinar función de detección según nombre de columna
    if 'fecha' in nombre_lower or 'date' in nombre_lower or '_date' in col:
        formatos = df[col].apply(detectar_formato_fecha)
        tipo_campo = 'FECHA'
    elif 'hora' in nombre_lower or 'time' in nombre_lower or '_time' in col:
        formatos = df[col].apply(detectar_formato_hora)
        tipo_campo = 'HORA'
    elif 'timestamp' in nombre_lower:
        formatos = df[col].apply(detectar_formato_fecha)
        tipo_campo = 'TIMESTAMP'
    else:
        formatos = df[col].apply(detectar_formato_generico)
        tipo_campo = 'GENERICO'
    
    # Contar formatos únicos
    conteo_formatos = formatos.value_counts()
    
    # Si hay más de 1 formato (excluyendo NULL y PLACEHOLDER), hay inconsistencia
    formatos_significativos = [f for f in conteo_formatos.index 
                               if f not in ['NULL', 'PLACEHOLDER', 'NO_FECHA', 'NO_HORA', 'NO_GENERICO']]
    
    if len(formatos_significativos) > 1 or (len(formatos_significativos) == 1 and len(conteo_formatos) > 2):
        problemas_formato[col] = {
            'tipo': tipo_campo,
            'formatos': conteo_formatos.to_dict(),
            'num_formatos': len(conteo_formatos)
        }

# Mostrar resultados
if problemas_formato:
    print(f"\n⚠️  SE DETECTARON INCONSISTENCIAS DE FORMATO EN {len(problemas_formato)} COLUMNA(S)\n")
    
    # Para cada columna con problemas
    for col, info in problemas_formato.items():
        print("="*80)
        print(f"📊 COLUMNA: {col}")
        print(f"   Tipo detectado: {info['tipo']}")
        print(f"   Número de formatos diferentes: {info['num_formatos']}")
        print("-"*80)
        
        # Crear DataFrame de distribución de formatos
        formatos_df = pd.DataFrame.from_dict(info['formatos'], orient='index', columns=['Cantidad'])
        formatos_df['Porcentaje'] = (formatos_df['Cantidad'] / len(df) * 100).round(2)
        formatos_df = formatos_df.sort_values('Cantidad', ascending=False)
        formatos_df.index.name = 'Formato'
        
        print("\nDistribución de formatos:")
        display(formatos_df)
        
        # Mostrar ejemplos de cada formato
        print("\nEjemplos de cada formato:")
        for formato in formatos_df.index[:10]:  # Mostrar máximo 10 formatos
            if info['tipo'] == 'FECHA':
                mask = df[col].apply(detectar_formato_fecha) == formato
            elif info['tipo'] in ['HORA', 'TIMESTAMP']:
                mask = df[col].apply(detectar_formato_hora) == formato
            else:
                mask = df[col].apply(detectar_formato_generico) == formato
            
            ejemplos = df.loc[mask, col].head(3).tolist()
            print(f"  • {formato}: {ejemplos}")
        
        # Gráfico de barras para esta columna
        if len(formatos_df) <= 15:  # Solo graficar si no hay demasiados formatos
            plt.figure(figsize=(12, 6))
            
            colores = plt.cm.Set3(range(len(formatos_df)))
            barras = plt.bar(range(len(formatos_df)), formatos_df['Cantidad'], 
                           color=colores, edgecolor='black', linewidth=1.2)
            
            # Agregar valores
            for i, (barra, valor, pct) in enumerate(zip(barras, formatos_df['Cantidad'], formatos_df['Porcentaje'])):
                plt.text(barra.get_x() + barra.get_width()/2, barra.get_height(),
                        f'{int(valor)}\n({pct}%)',
                        ha='center', va='bottom', fontsize=9, fontweight='bold')
            
            plt.xlabel('Tipo de formato', fontsize=11, fontweight='bold')
            plt.ylabel('Cantidad de registros', fontsize=11, fontweight='bold')
            plt.title(f'Distribución de Formatos en columna: {col}\n({info["tipo"]})', 
                     fontsize=13, fontweight='bold', pad=15)
            plt.xticks(range(len(formatos_df)), formatos_df.index, rotation=45, ha='right')
            plt.grid(axis='y', alpha=0.3, linestyle='--')
            plt.tight_layout()
            plt.show()
        
        print("\n")
    
    # Resumen general
    print("="*80)
    print("RESUMEN GENERAL DE INCONSISTENCIAS")
    print("="*80)
    
    resumen_general = pd.DataFrame([
        {
            'Columna': col,
            'Tipo': info['tipo'],
            'Formatos diferentes': info['num_formatos'],
            'Formato principal': max(info['formatos'], key=info['formatos'].get),
            'Registros formato principal': max(info['formatos'].values())
        }
        for col, info in problemas_formato.items()
    ])
    
    display(resumen_general)
    
    # Recomendaciones
    print("\n" + "="*80)
    print("RECOMENDACIONES")
    print("="*80)
    print("1. Estandarizar formatos de fecha al formato ISO 8601 (YYYY-MM-DD)")
    print("2. Usar pd.to_datetime() con parámetros para manejar formatos mixtos:")
    print("   df['columna'] = pd.to_datetime(df['columna'], format='%d/%m/%y', errors='coerce')")
    print("3. Para fechas con múltiples formatos, usar infer_datetime_format=True:")
    print("   df['columna'] = pd.to_datetime(df['columna'], infer_datetime_format=True, errors='coerce')")
    print("4. Identificar y corregir valores '-' o placeholders antes de conversión")
    print("5. Validar que todas las fechas convertidas sean coherentes")
    print("="*80)
    
else:
    print("\n✅ NO SE DETECTARON INCONSISTENCIAS DE FORMATO")
    print("\nTodas las columnas tienen formatos consistentes.")
    print("="*80)


In [ ]:
# --- Errores categóricos ---
print("\nValores únicos por columna categórica:")
for col in df.select_dtypes(include="object").columns:
    print(f"{col}: {df[col].unique()[:10]} ...")



In [ ]:
# ============================================
# --- Detección de espacios adicionales en categorías ---
# ============================================

import re

print("\n" + "="*80)
print("ANÁLISIS DE ESPACIOS ADICIONALES EN CATEGORÍAS")
print("="*80)
print("(Detectando espacios al inicio, final o múltiples que causan inconsistencias)\n")

# Función para analizar espacios en valores
def analizar_espacios(valor):
    """Analiza si un valor tiene espacios problemáticos"""
    if pd.isnull(valor):
        return None
    
    valor_str = str(valor)
    problemas = {
        'leading': len(valor_str) - len(valor_str.lstrip()),
        'trailing': len(valor_str) - len(valor_str.rstrip()),
        'multiple': len(re.findall(r'\s{2,}', valor_str)),
        'valor_limpio': valor_str.strip(),
        'longitud_original': len(valor_str),
        'longitud_limpia': len(valor_str.strip())
    }
    return problemas

# Analizar cada columna categórica
problemas_espacios = {}

for col in df.select_dtypes(include="object").columns:
    # Analizar espacios en cada valor
    analisis = df[col].apply(analizar_espacios)
    
    # Filtrar valores con problemas (tienen espacios adicionales)
    valores_con_problemas = df[col][
        analisis.apply(lambda x: x is not None and 
                      (x['leading'] > 0 or x['trailing'] > 0 or x['multiple'] > 0))
    ]
    
    if len(valores_con_problemas) > 0:
        # Agrupar valores por su versión limpia
        valores_limpios = valores_con_problemas.apply(lambda x: str(x).strip())
        
        # Encontrar casos donde el mismo valor limpio tiene múltiples representaciones
        grupos_duplicados = {}
        for valor_limpio in valores_limpios.unique():
            # Encontrar todas las variantes con espacios de este valor
            variantes = valores_con_problemas[valores_limpios == valor_limpio].unique()
            if len(variantes) > 1 or any(v != valor_limpio for v in variantes):
                grupos_duplicados[valor_limpio] = {
                    'variantes': list(variantes),
                    'longitudes': [len(v) for v in variantes],
                    'total_registros': len(valores_con_problemas[valores_limpios == valor_limpio])
                }
        
        if grupos_duplicados or len(valores_con_problemas) > 0:
            problemas_espacios[col] = {
                'total_afectados': len(valores_con_problemas),
                'porcentaje': (len(valores_con_problemas) / len(df)) * 100,
                'grupos_duplicados': grupos_duplicados,
                'leading_count': sum(analisis.apply(lambda x: x['leading'] if x else 0) > 0),
                'trailing_count': sum(analisis.apply(lambda x: x['trailing'] if x else 0) > 0),
                'multiple_count': sum(analisis.apply(lambda x: x['multiple'] if x else 0) > 0)
            }

# Mostrar resultados
if problemas_espacios:
    print(f"⚠️  SE DETECTARON PROBLEMAS DE ESPACIOS EN {len(problemas_espacios)} COLUMNA(S)\n")
    
    # Resumen general
    resumen_data = []
    for col, info in problemas_espacios.items():
        resumen_data.append({
            'Columna': col,
            'Registros afectados': info['total_afectados'],
            'Porcentaje (%)': round(info['porcentaje'], 2),
            'Espacios al inicio': info['leading_count'],
            'Espacios al final': info['trailing_count'],
            'Espacios múltiples': info['multiple_count'],
            'Valores duplicados': len(info['grupos_duplicados'])
        })
    
    resumen_df = pd.DataFrame(resumen_data)
    resumen_df = resumen_df.sort_values('Registros afectados', ascending=False)
    
    print("RESUMEN GENERAL:")
    print("-"*80)
    display(resumen_df)
    
    # Gráfico de resumen
    if len(problemas_espacios) > 0:
        fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(16, 6))
        
        # Gráfico 1: Registros afectados por columna
        cols = resumen_df['Columna'].tolist()
        valores = resumen_df['Registros afectados'].tolist()
        colores = plt.cm.Reds(np.linspace(0.4, 0.9, len(cols)))
        
        barras = ax1.barh(cols, valores, color=colores, edgecolor='black', linewidth=1.2)
        for barra, valor, pct in zip(barras, valores, resumen_df['Porcentaje (%)']):
            ax1.text(barra.get_width(), barra.get_y() + barra.get_height()/2,
                    f' {int(valor)} ({pct}%)',
                    va='center', fontweight='bold', fontsize=9)
        
        ax1.set_xlabel('Número de registros con espacios adicionales', fontsize=11, fontweight='bold')
        ax1.set_ylabel('Columna', fontsize=11, fontweight='bold')
        ax1.set_title('Registros Afectados por Espacios Adicionales', fontsize=12, fontweight='bold')
        ax1.grid(axis='x', alpha=0.3, linestyle='--')
        
        # Gráfico 2: Tipo de problema
        tipos_problemas = resumen_df[['Espacios al inicio', 'Espacios al final', 'Espacios múltiples']].sum()
        colores_tipos = ['#ff6b6b', '#feca57', '#48dbfb']
        
        barras2 = ax2.bar(tipos_problemas.index, tipos_problemas.values, 
                         color=colores_tipos, edgecolor='black', linewidth=1.2)
        for barra, valor in zip(barras2, tipos_problemas.values):
            ax2.text(barra.get_x() + barra.get_width()/2, barra.get_height(),
                    f'{int(valor)}',
                    ha='center', va='bottom', fontweight='bold', fontsize=11)
        
        ax2.set_ylabel('Cantidad de registros', fontsize=11, fontweight='bold')
        ax2.set_title('Distribución por Tipo de Problema', fontsize=12, fontweight='bold')
        ax2.set_xticklabels(tipos_problemas.index, rotation=15, ha='right')
        ax2.grid(axis='y', alpha=0.3, linestyle='--')
        
        plt.tight_layout()
        plt.show()
    
    # Detalles por columna
    print("\n" + "="*80)
    print("DETALLES POR COLUMNA")
    print("="*80)
    
    for col, info in problemas_espacios.items():
        print(f"\n{'='*80}")
        print(f"📋 COLUMNA: {col}")
        print(f"   Total registros afectados: {info['total_afectados']} ({info['porcentaje']:.2f}%)")
        print(f"   Espacios al inicio: {info['leading_count']}")
        print(f"   Espacios al final: {info['trailing_count']}")
        print(f"   Espacios múltiples internos: {info['multiple_count']}")
        print("-"*80)
        
        # Mostrar grupos duplicados (mismo valor con diferentes espacios)
        if info['grupos_duplicados']:
            print(f"\n   ⚠️  VALORES DUPLICADOS POR ESPACIOS ({len(info['grupos_duplicados'])} casos):\n")
            
            # Limitar a mostrar los primeros 10 grupos
            for i, (valor_limpio, detalle) in enumerate(list(info['grupos_duplicados'].items())[:10], 1):
                print(f"   {i}. Valor base: '{valor_limpio}' (longitud: {len(valor_limpio)})")
                print(f"      Afecta a {detalle['total_registros']} registros")
                print(f"      Variantes encontradas ({len(detalle['variantes'])}):")
                
                for j, (variante, longitud) in enumerate(zip(detalle['variantes'], detalle['longitudes']), 1):
                    # Representación visual de los espacios
                    repr_variante = repr(variante)
                    espacios_inicio = len(variante) - len(variante.lstrip())
                    espacios_final = len(variante) - len(variante.rstrip())
                    
                    indicador = ""
                    if espacios_inicio > 0:
                        indicador += f"←{espacios_inicio} espacio(s) al inicio "
                    if espacios_final > 0:
                        indicador += f"→{espacios_final} espacio(s) al final "
                    
                    print(f"         {j}. {repr_variante} [Long: {longitud}] {indicador}")
                print()
            
            if len(info['grupos_duplicados']) > 10:
                print(f"   ... y {len(info['grupos_duplicados']) - 10} grupos más\n")
        
        # Mostrar ejemplos de registros afectados
        print("   Ejemplos de valores con espacios adicionales:")
        valores_ejemplo = df[col][
            df[col].apply(lambda x: x is not None and 
                         (len(str(x)) != len(str(x).strip()) if x is not None else False))
        ].head(5)
        
        for idx, valor in enumerate(valores_ejemplo, 1):
            print(f"      {idx}. {repr(valor)} → limpio: {repr(str(valor).strip())}")
        print()
    
    # Recomendaciones
    print("="*80)
    print("RECOMENDACIONES")
    print("="*80)
    print("1. Limpiar espacios usando el método .strip():")
    print("   df['columna'] = df['columna'].str.strip()")
    print("\n2. Para múltiples espacios internos, usar regex:")
    print("   df['columna'] = df['columna'].str.replace(r'\\s+', ' ', regex=True)")
    print("\n3. Aplicar limpieza a todas las columnas de texto:")
    print("   for col in df.select_dtypes(include='object').columns:")
    print("       df[col] = df[col].str.strip()")
    print("\n4. Verificar después de limpiar que no se crearon duplicados")
    print("="*80)
    
else:
    print("✅ NO SE DETECTARON PROBLEMAS DE ESPACIOS ADICIONALES")
    print("\nTodas las columnas categóricas tienen valores sin espacios problemáticos.")
    print("="*80)


In [ ]:
from scipy.stats import zscore
# --- Outliers ---
num_cols = df.select_dtypes(include=np.number).columns
z_scores = pd.DataFrame(zscore(df[num_cols], nan_policy='omit'), columns=num_cols)
outliers_mask = (np.abs(z_scores) > 3)
outliers_count = outliers_mask.sum()
print("\nDetección de outliers (z-score > 3):")
print(outliers_count)


In [ ]:

# --- Campos redundantes ---
low_var_cols = [col for col in df.columns if df[col].nunique() <= 1]
print("\nColumnas con baja varianza (posibles redundantes):")
print(low_var_cols)



In [ ]:
# ============================================
# --- Codificación incorrecta ---
# ============================================

import re

print("\n" + "="*80)
print("ANÁLISIS DE PROBLEMAS DE CODIFICACIÓN (ENCODING)")
print("="*80)

# Función para detectar problemas de encoding
def detectar_problemas_encoding(texto):
    """Detecta varios tipos de problemas de encoding en un texto"""
    if pd.isnull(texto):
        return []
    
    texto_str = str(texto)
    problemas = []
    
    # 1. Detectar caracter de reemplazo (�)
    if '�' in texto_str:
        problemas.append('replacement_char')
    
    # 2. Detectar secuencias UTF-8 mal decodificadas (como Ã©, Ã±, etc.)
    if re.search(r'[ÃÂ][^a-zA-Z0-9\s]', texto_str):
        problemas.append('utf8_malformed')
    
    # 3. Detectar caracteres de control (no imprimibles)
    if re.search(r'[\x00-\x08\x0b-\x0c\x0e-\x1f\x7f-\x9f]', texto_str):
        problemas.append('control_chars')
    
    # 4. Detectar mezcla sospechosa de caracteres latinos y otros
    if re.search(r'[^\x00-\x7F\u00C0-\u017F\u0020-\u007E\u00A0-\u00FF]', texto_str):
        problemas.append('mixed_encoding')
    
    # 5. Detectar espacios duplicados o extraños
    if re.search(r'\s{3,}|[\u00A0\u2000-\u200B]', texto_str):
        problemas.append('weird_spaces')
    
    return problemas

# Analizar cada columna de texto
resultados_encoding = {}
ejemplos_problemas = {}

for col in df.select_dtypes(include="object").columns:
    # Detectar problemas en cada valor
    problemas_por_fila = df[col].apply(detectar_problemas_encoding)
    
    # Contar filas con problemas
    filas_con_problemas = problemas_por_fila.apply(lambda x: len(x) > 0).sum()
    
    if filas_con_problemas > 0:
        # Guardar estadísticas
        resultados_encoding[col] = {
            'total_problemas': filas_con_problemas,
            'porcentaje': (filas_con_problemas / len(df)) * 100
        }
        
        # Guardar ejemplos de valores con problemas
        indices_problemas = problemas_por_fila[problemas_por_fila.apply(lambda x: len(x) > 0)].index[:5]
        ejemplos_problemas[col] = df.loc[indices_problemas, col].tolist()

# Mostrar resultados
if resultados_encoding:
    print(f"\n⚠️  SE DETECTARON PROBLEMAS DE ENCODING EN {len(resultados_encoding)} COLUMNA(S)\n")
    
    # Crear DataFrame resumen
    resumen_df = pd.DataFrame.from_dict(resultados_encoding, orient='index')
    resumen_df = resumen_df.sort_values('total_problemas', ascending=False)
    resumen_df['porcentaje'] = resumen_df['porcentaje'].round(2)
    resumen_df.columns = ['Registros afectados', 'Porcentaje (%)']
    
    print("RESUMEN DE PROBLEMAS POR COLUMNA:")
    print("-" * 80)
    display(resumen_df)
    
    # Gráfico de barras
    if len(resultados_encoding) > 0:
        plt.figure(figsize=(12, 6))
        cols_plot = resumen_df.index.tolist()
        valores_plot = resumen_df['Registros afectados'].tolist()
        
        colores = ['#ff6b6b' if v > len(df)*0.1 else '#feca57' if v > len(df)*0.01 else '#48dbfb' 
                   for v in valores_plot]
        
        barras = plt.barh(cols_plot, valores_plot, color=colores, edgecolor='black', linewidth=1.2)
        
        # Agregar valores en las barras
        for i, (barra, valor) in enumerate(zip(barras, valores_plot)):
            porcentaje = (valor / len(df)) * 100
            plt.text(barra.get_width(), barra.get_y() + barra.get_height()/2,
                    f' {int(valor)} ({porcentaje:.1f}%)',
                    va='center', fontweight='bold', fontsize=10)
        
        plt.xlabel('Número de registros con problemas de encoding', fontsize=11, fontweight='bold')
        plt.ylabel('Columna', fontsize=11, fontweight='bold')
        plt.title('Problemas de Encoding por Columna\n(Rojo: >10% | Amarillo: >1% | Azul: <1%)', 
                 fontsize=13, fontweight='bold', pad=15)
        plt.grid(axis='x', alpha=0.3, linestyle='--')
        plt.tight_layout()
        plt.show()
    
    # Mostrar ejemplos detallados
    print("\n" + "="*80)
    print("EJEMPLOS DE VALORES CON PROBLEMAS DE ENCODING")
    print("="*80)
    
    for col, ejemplos in ejemplos_problemas.items():
        print(f"\n📋 Columna: {col}")
        print(f"   Registros afectados: {resultados_encoding[col]['total_problemas']} ({resultados_encoding[col]['porcentaje']:.2f}%)")
        print(f"   Ejemplos de valores con problemas:")
        for i, ejemplo in enumerate(ejemplos, 1):
            # Mostrar representación del valor
            repr_valor = repr(ejemplo)
            if len(repr_valor) > 80:
                repr_valor = repr_valor[:77] + "..."
            print(f"      {i}. {repr_valor}")
        print("-" * 80)
    
    # Recomendaciones
    print("\n" + "="*80)
    print("RECOMENDACIONES")
    print("="*80)
    print("1. Verificar el encoding del archivo fuente (UTF-8, ISO-8859-1, etc.)")
    print("2. Re-cargar el CSV especificando el encoding correcto:")
    print("   Ejemplo: pd.read_csv('archivo.csv', encoding='latin-1')")
    print("3. Considerar usar ftfy (fix text for you) para corregir automáticamente:")
    print("   pip install ftfy")
    print("   from ftfy import fix_text")
    print("   df[col] = df[col].apply(lambda x: fix_text(x) if pd.notnull(x) else x)")
    print("="*80)
    
else:
    print("\n✅ NO SE DETECTARON PROBLEMAS DE ENCODING")
    print("\nTodas las columnas de texto tienen codificación correcta.")
    print("\nSe verificaron los siguientes problemas comunes:")
    print("  • Caracteres de reemplazo (�)")
    print("  • Secuencias UTF-8 mal decodificadas")
    print("  • Caracteres de control no imprimibles")
    print("  • Mezcla de encodings diferentes")
    print("  • Espacios extraños o duplicados")
    print("="*80)


In [ ]:
# ============================================
# 4. Visualizaciones rápidas en la celda 13 
# ============================================

# Mapa de valores nulos
sns.heatmap(df.isnull(), cbar=False)
plt.title("Mapa de valores nulos")
plt.show()

# Boxplot solo para columnas numéricas con datos suficientes
for col in num_cols:
    if df[col].dropna().nunique() > 1:  # al menos dos valores distintos
        plt.figure(figsize=(6,3))
        sns.boxplot(x=df[col])
        plt.title(f"Outliers en {col}")
        plt.show()
    else:
        print(f"⚠️ Columna '{col}' omitida: no tiene suficientes datos numéricos válidos para graficar.")


In [ ]:

# ============================================
# 5. Reporte final
# ============================================

print("\n\n--- REPORTE DE PROBLEMAS DETECTADOS ---")
print("1. Valores nulos detectados en:", df.columns[df.isnull().any()].tolist())
print("2. Filas duplicadas:", df.duplicated().sum())
print("3. Columnas con problemas de formato revisadas manualmente.")
print("4. Columnas con outliers en columnas numéricas:", outliers_count[outliers_count > 0].index.tolist())
print("5. Columnas redundantes:", low_var_cols)


In [ ]:
import re
import unicodedata
from difflib import get_close_matches

# ============================================
# 6. Validación de caracteres especiales y errores de escritura
# ============================================

# --- 6.1 Detectar caracteres especiales ---
print("\nCaracteres especiales detectados en columnas de texto:")
for col in df.select_dtypes(include="object").columns:
    especiales = df[col].dropna().apply(lambda x: re.findall(r"[^a-zA-Z0-9\sáéíóúÁÉÍÓÚñÑ]", str(x)))
    especiales = [c for sub in especiales for c in sub]
    if especiales:
        print(f"Columna {col}: {set(especiales)}")

# --- 6.2 Normalización de texto ---
def normalizar_texto(texto):
    if pd.isnull(texto):
        return texto
    # Pasar a minúsculas
    texto = texto.lower().strip()
    # Quitar acentos
    texto = ''.join(
        c for c in unicodedata.normalize('NFD', texto)
        if unicodedata.category(c) != 'Mn'
    )
    return texto

for col in df.select_dtypes(include="object").columns:
    df[col] = df[col].apply(normalizar_texto)

# --- 6.3 Detectar variantes de categorías ---
print("\nPosibles errores de categorización (formas diferentes de lo mismo):")
for col in df.select_dtypes(include="object").columns:
    valores = df[col].dropna().unique()
    if len(valores) < 50:  # solo columnas con pocas categorías
        print(f"\nColumna {col}:")
        for v in valores:
            similares = get_close_matches(v, valores, cutoff=0.8)
            if len(similares) > 1:
                print(f"  '{v}' ~ {similares}")


In [ ]:
# Valores únicos con su frecuencia en formato DataFrame
# df_unicos_conteo = df["AddressType"].value_counts().reset_index()
# df_unicos_conteo.columns = ["AddressType", "conteo"]
# display(df_unicos_conteo)


In [ ]:
# ============================================
# --- Detección de caracteres especiales atípicos ---
# ============================================

import re
import unicodedata

print("\n" + "="*80)
print("ANÁLISIS DE CARACTERES ESPECIALES ATÍPICOS")
print("="*80)
print("(Detectando caracteres no estándar que pueden causar problemas)\n")

# Definir caracteres permitidos/comunes
# Ajustar según necesidades del dataset
CARACTERES_COMUNES = set(
    'abcdefghijklmnopqrstuvwxyz'
    'ABCDEFGHIJKLMNOPQRSTUVWXYZ'
    'áéíóúÁÉÍÓÚñÑüÜ'  # Acentos españoles
    '0123456789'
    ' .,;:/-_()[]{}@#$%&*+=<>?!"\'\n\t'  # Puntuación común
)

def detectar_caracteres_atipicos(valor):
    """Detecta caracteres atípicos en un valor"""
    if pd.isnull(valor):
        return None
    
    valor_str = str(valor)
    caracteres_atipicos = []
    
    for char in valor_str:
        if char not in CARACTERES_COMUNES:
            # Obtener información del caracter
            try:
                nombre_unicode = unicodedata.name(char, 'DESCONOCIDO')
                categoria = unicodedata.category(char)
            except:
                nombre_unicode = 'ERROR'
                categoria = 'UNKNOWN'
            
            caracteres_atipicos.append({
                'char': char,
                'codigo': ord(char),
                'hex': hex(ord(char)),
                'nombre': nombre_unicode,
                'categoria': categoria
            })
    
    return caracteres_atipicos if caracteres_atipicos else None

# Analizar todas las columnas de texto
problemas_caracteres = {}

for col in df.select_dtypes(include="object").columns:
    # Detectar caracteres atípicos en cada valor
    caracteres_por_valor = df[col].apply(detectar_caracteres_atipicos)
    
    # Filtrar valores con caracteres atípicos
    valores_con_atipicos = caracteres_por_valor[caracteres_por_valor.notna()]
    
    if len(valores_con_atipicos) > 0:
        # Recopilar todos los caracteres únicos encontrados
        chars_unicos = {}
        ejemplos_valores = []
        
        for idx, lista_chars in valores_con_atipicos.items():
            if lista_chars:
                # Guardar ejemplo del valor completo
                if len(ejemplos_valores) < 10:
                    ejemplos_valores.append({
                        'valor': df.loc[idx, col],
                        'caracteres': [c['char'] for c in lista_chars]
                    })
                
                # Agrupar caracteres únicos
                for char_info in lista_chars:
                    char = char_info['char']
                    if char not in chars_unicos:
                        chars_unicos[char] = {
                            'info': char_info,
                            'count': 0,
                            'ejemplos': []
                        }
                    chars_unicos[char]['count'] += 1
                    if len(chars_unicos[char]['ejemplos']) < 3:
                        chars_unicos[char]['ejemplos'].append(df.loc[idx, col])
        
        problemas_caracteres[col] = {
            'total_valores_afectados': len(valores_con_atipicos),
            'porcentaje': (len(valores_con_atipicos) / len(df)) * 100,
            'caracteres_unicos': chars_unicos,
            'ejemplos_valores': ejemplos_valores
        }

# Mostrar resultados
if problemas_caracteres:
    print(f"⚠️  SE DETECTARON CARACTERES ATÍPICOS EN {len(problemas_caracteres)} COLUMNA(S)\n")
    
    # Resumen general
    print("="*80)
    print("RESUMEN GENERAL")
    print("="*80)
    
    resumen_data = []
    for col, info in problemas_caracteres.items():
        resumen_data.append({
            'Columna': col,
            'Valores afectados': info['total_valores_afectados'],
            'Porcentaje (%)': round(info['porcentaje'], 2),
            'Caracteres únicos': len(info['caracteres_unicos'])
        })
    
    resumen_df = pd.DataFrame(resumen_data)
    resumen_df = resumen_df.sort_values('Valores afectados', ascending=False)
    display(resumen_df)
    
    # Gráfico de resumen
    if len(problemas_caracteres) > 0:
        fig, ax = plt.subplots(figsize=(14, 6))
        
        cols = resumen_df['Columna'].tolist()
        valores = resumen_df['Valores afectados'].tolist()
        colores = plt.cm.Oranges(np.linspace(0.4, 0.9, len(cols)))
        
        barras = ax.barh(cols, valores, color=colores, edgecolor='black', linewidth=1.2)
        
        for barra, valor, pct, chars in zip(barras, valores, resumen_df['Porcentaje (%)'], 
                                            resumen_df['Caracteres únicos']):
            ax.text(barra.get_width(), barra.get_y() + barra.get_height()/2,
                    f' {int(valor)} valores ({pct}%) | {chars} chars únicos',
                    va='center', fontweight='bold', fontsize=9)
        
        ax.set_xlabel('Valores con caracteres atípicos', fontsize=11, fontweight='bold')
        ax.set_ylabel('Columna', fontsize=11, fontweight='bold')
        ax.set_title('Detección de Caracteres Especiales Atípicos por Columna', 
                     fontsize=13, fontweight='bold', pad=15)
        ax.grid(axis='x', alpha=0.3, linestyle='--')
        plt.tight_layout()
        plt.show()
    
    # Detalles por columna
    print("\n" + "="*80)
    print("DETALLES POR COLUMNA")
    print("="*80)
    
    for col, info in problemas_caracteres.items():
        print(f"\n{'='*80}")
        print(f"📋 COLUMNA: {col}")
        print(f"   Valores afectados: {info['total_valores_afectados']} ({info['porcentaje']:.2f}%)")
        print(f"   Caracteres únicos atípicos encontrados: {len(info['caracteres_unicos'])}")
        print("-"*80)
        
        # Tabla de caracteres únicos
        print("\n   CARACTERES ATÍPICOS DETECTADOS:\n")
        
        chars_tabla = []
        for char, data in sorted(info['caracteres_unicos'].items(), 
                                key=lambda x: x[1]['count'], reverse=True):
            char_info = data['info']
            chars_tabla.append({
                'Caracter': f"'{char}'" if char.isprintable() else '[No visible]',
                'Código': f"U+{char_info['hex'][2:].upper().zfill(4)}",
                'Decimal': char_info['codigo'],
                'Nombre Unicode': char_info['nombre'][:40],
                'Categoría': char_info['categoria'],
                'Ocurrencias': data['count']
            })
        
        chars_df = pd.DataFrame(chars_tabla)
        display(chars_df.head(20))  # Mostrar hasta 20 caracteres
        
        if len(chars_tabla) > 20:
            print(f"\n   ... y {len(chars_tabla) - 20} caracteres más\n")
        
        # Ejemplos de valores afectados
        print("\n   EJEMPLOS DE VALORES CON CARACTERES ATÍPICOS:\n")
        for i, ejemplo in enumerate(info['ejemplos_valores'][:5], 1):
            valor = ejemplo['valor']
            chars = ejemplo['caracteres']
            
            # Resaltar caracteres atípicos en el valor
            valor_repr = repr(valor)
            if len(valor_repr) > 100:
                valor_repr = valor_repr[:97] + "...'"
            
            print(f"   {i}. {valor_repr}")
            print(f"      → Caracteres atípicos: {chars}")
            print(f"      → Códigos Unicode: {[f'U+{hex(ord(c))[2:].upper().zfill(4)}' for c in chars]}")
            print()
        
        print("-"*80)
    
    # Categorías de caracteres más comunes
    print("\n" + "="*80)
    print("ANÁLISIS DE CATEGORÍAS UNICODE")
    print("="*80)
    
    todas_categorias = {}
    for col, info in problemas_caracteres.items():
        for char_data in info['caracteres_unicos'].values():
            cat = char_data['info']['categoria']
            if cat not in todas_categorias:
                todas_categorias[cat] = {'count': 0, 'descripcion': ''}
            todas_categorias[cat]['count'] += char_data['count']
    
    # Descripciones de categorías comunes
    descripciones_cat = {
        'Cc': 'Caracteres de control',
        'Cf': 'Caracteres de formato',
        'Cn': 'No asignados',
        'Co': 'Uso privado',
        'Cs': 'Sustitutos',
        'Ll': 'Letra minúscula',
        'Lm': 'Letra modificadora',
        'Lo': 'Otra letra',
        'Lt': 'Letra título',
        'Lu': 'Letra mayúscula',
        'Mc': 'Marca de espaciado',
        'Me': 'Marca adjunta',
        'Mn': 'Marca sin espaciado',
        'Nd': 'Número decimal',
        'Nl': 'Número letra',
        'No': 'Otro número',
        'Pc': 'Puntuación conectora',
        'Pd': 'Puntuación guión',
        'Pe': 'Puntuación cierre',
        'Pf': 'Puntuación final',
        'Pi': 'Puntuación inicial',
        'Po': 'Otra puntuación',
        'Ps': 'Puntuación apertura',
        'Sc': 'Símbolo moneda',
        'Sk': 'Símbolo modificador',
        'Sm': 'Símbolo matemático',
        'So': 'Otro símbolo',
        'Zl': 'Separador línea',
        'Zp': 'Separador párrafo',
        'Zs': 'Separador espacio'
    }
    
    print("\nDistribución por categoría Unicode:")
    for cat, data in sorted(todas_categorias.items(), key=lambda x: x[1]['count'], reverse=True):
        desc = descripciones_cat.get(cat, 'Desconocido')
        print(f"  • {cat} ({desc}): {data['count']} ocurrencias")
    
    # Recomendaciones
    print("\n" + "="*80)
    print("RECOMENDACIONES")
    print("="*80)
    print("1. Revisar si los caracteres atípicos son legítimos o errores de encoding")
    print("\n2. Para eliminar caracteres no ASCII:")
    print("   df['columna'] = df['columna'].str.encode('ascii', 'ignore').str.decode('ascii')")
    print("\n3. Para normalizar caracteres Unicode (quitar acentos):")
    print("   import unicodedata")
    print("   def normalizar(texto):")
    print("       return ''.join(c for c in unicodedata.normalize('NFD', texto)")
    print("                      if unicodedata.category(c) != 'Mn')")
    print("\n4. Para reemplazar caracteres específicos:")
    print("   df['columna'] = df['columna'].str.replace('[caracter]', '', regex=False)")
    print("\n5. Verificar el encoding del archivo original (UTF-8, Latin-1, etc.)")
    print("="*80)
    
else:
    print("✅ NO SE DETECTARON CARACTERES ATÍPICOS")
    print("\nTodas las columnas contienen solo caracteres estándar.")
    print("\nCaracteres considerados estándar:")
    print("  • Letras: a-z, A-Z")
    print("  • Acentos españoles: á, é, í, ó, ú, ñ, ü")
    print("  • Números: 0-9")
    print("  • Puntuación común: . , ; : / - _ ( ) [ ] { } @ # $ % & * + = < > ? ! \" '")
    print("="*80)


In [ ]:
# Filtrar filas donde la columna 'AddressType' contenga 'interse'
# df_filtrado = df[df["AddressType"].str.contains("interse", case=False, na=False)]

# Mostrar solo la columna filtrada
#  print("\nValores únicos en AddressType filtrados:")

# Convertir los valores únicos en un DataFrame
# df_unicos = pd.DataFrame(df_filtrado["AddressType"].unique(), columns=["AddressType"])
# display(df_unicos)




In [ ]:
import re
import unicodedata
from difflib import get_close_matches

# --- Función de normalización básica ---
def normalizar_texto(texto):
    if pd.isnull(texto):
        return texto
    texto = texto.lower().strip()
    texto = ''.join(
        c for c in unicodedata.normalize('NFD', texto)
        if unicodedata.category(c) != 'Mn'
    )
    # quitar caracteres especiales no deseados
    texto = re.sub(r"[^a-z0-9\s]", "", texto)
    return texto

# --- Función para unificar categorías similares ---
def unificar_categorias(serie, cutoff=0.8):
    valores = serie.dropna().unique()
    mapping = {}
    for v in valores:
        similares = get_close_matches(v, valores, cutoff=cutoff)
        if len(similares) > 1:
            # elegir el más frecuente como "estándar"
            freq = serie.value_counts()
            estandar = max(similares, key=lambda x: freq.get(x, 0))
            for s in similares:
                mapping[s] = estandar
    # aplicar reemplazo
    return serie.replace(mapping)

# --- Aplicar a todas las columnas de texto ---
for col in df.select_dtypes(include="object").columns:
    # normalizar texto
    df[col] = df[col].apply(normalizar_texto)
    # unificar categorías similares
    df[col] = unificar_categorias(df[col])

# --- Reporte final ---
print("\n--- SOLUCIÓN APLICADA ---")
print("1. Texto normalizado (minúsculas, sin acentos, sin caracteres especiales).")
print("2. Categorías similares unificadas automáticamente según frecuencia.")
print("3. Aplicado en todas las columnas de texto.")


In [ ]:
import re
import unicodedata
from difflib import get_close_matches

# ============================================
# 6. Aplicación de soluciones genéricas
# ============================================

# Configuración
config = {
    "imputacion_numerica": "media",
    "imputacion_categorica": "moda",
    "formato_fecha": "%Y-%m-%d",
    "deduplicar": True,
    "validacion_rangos": {
        "edad": (0, 120),
        "precio": (0, 10000)
    },
    "conversion_unidades": {
        "distancia_km": ("distancia_millas", 0.621371)
    }
}

# --- 6.1 Imputación de valores nulos ---
for col in df.select_dtypes(include=np.number).columns:
    if df[col].isnull().sum() > 0:
        if config["imputacion_numerica"] == "media":
            df[col].fillna(df[col].mean(), inplace=True)
        elif config["imputacion_numerica"] == "mediana":
            df[col].fillna(df[col].median(), inplace=True)
        elif config["imputacion_numerica"] == "cero":
            df[col].fillna(0, inplace=True)

for col in df.select_dtypes(include="object").columns:
    if df[col].isnull().sum() > 0:
        if config["imputacion_categorica"] == "moda":
            df[col].fillna(df[col].mode()[0], inplace=True)
        elif config["imputacion_categorica"] == "desconocido":
            df[col].fillna("desconocido", inplace=True)

# --- 6.2 Normalización de fechas ---
for col in df.columns:
    if "fecha" in col.lower():
        try:
            df[col] = pd.to_datetime(df[col], errors="coerce")
            df[col] = df[col].dt.strftime(config["formato_fecha"])
        except Exception as e:
            print(f"No se pudo normalizar {col}: {e}")

# --- 6.3 Deduplicación ---
if config["deduplicar"]:
    df.drop_duplicates(inplace=True)

# --- 6.4 Validación de rangos ---
for col, (min_val, max_val) in config["validacion_rangos"].items():
    if col in df.columns:
        df.loc[(df[col] < min_val) | (df[col] > max_val), col] = np.nan

# --- 6.5 Conversión de unidades ---
for col, (new_col, factor) in config["conversion_unidades"].items():
    if col in df.columns:
        df[new_col] = df[col] * factor

# --- 6.6 Normalización de categorías ---
def normalizar_texto(texto):
    if pd.isnull(texto): return texto
    texto = texto.lower().strip()
    texto = ''.join(c for c in unicodedata.normalize('NFD', texto) if unicodedata.category(c) != 'Mn')
    texto = re.sub(r"[^a-z0-9\s]", "", texto)
    return texto

# --- Función genérica para derivar valores desde otra columna ---
def derivar_valor(texto):
    if pd.isna(texto): return np.nan
    s = str(texto).lower().strip()
    # Ejemplo genérico: extraer número inicial
    m = re.match(r"^\s*(\d+)\b", s)
    if m: return int(m.group(1))
    # Ejemplo genérico: detectar intersecciones
    if "/" in s: return "intersection"
    return np.nan

# --- 1. Detectar columnas nulas o constantes ---
cols_full_null = [c for c in df.columns if df[c].isna().all()]
cols_constant = [c for c in df.columns if df[c].nunique(dropna=True) <= 1]

print("Columnas 100% nulas:", cols_full_null)
print("Columnas constantes:", cols_constant)

# --- 2. Aplicar derivación genérica ---
for col in cols_full_null:
    # Intentar derivar desde Address si existe
    if "Address" in df.columns:
        df[col] = df["Address"].apply(derivar_valor)
        df[f"has_{col}"] = df[col].notna()

# --- 3. Normalización de texto en todas las columnas categóricas ---
for col in df.select_dtypes(include="object").columns:
    df[col] = df[col].apply(normalizar_texto)

# --- 4. Unificación de categorías similares ---
def unificar_categorias(serie, cutoff=0.8):
    valores = serie.dropna().unique()
    mapping = {}
    for v in valores:
        similares = get_close_matches(v, valores, cutoff=cutoff)
        if len(similares) > 1:
            freq = serie.value_counts()
            estandar = max(similares, key=lambda x: freq.get(x, 0))
            for s in similares:
                mapping[s] = estandar
    return serie.replace(mapping)

for col in df.select_dtypes(include="object").columns:
    df[col] = unificar_categorias(df[col])

In [ ]:
# ============================================
# 7. Dataset limpio y reporte final
# ============================================

print("\n--- REPORTE DE SOLUCIONES APLICADAS ---")
print("1. Imputación de valores nulos realizada.")
print("2. Fechas normalizadas al formato:", config["formato_fecha"])
print("3. Deduplicación aplicada:", config["deduplicar"])
print("4. Validación de rangos aplicada en columnas:", list(config["validacion_rangos"].keys()))
print("5. Conversión de unidades aplicada en columnas:", list(config["conversion_unidades"].keys()))
print("6. Normalización de texto y categorías aplicada.")
print("7. Derivación de 'Range' desde 'Address' aplicada.")
print("8. Normalización semántica de 'AdressTy' aplicada.")
print("\nDimensiones finales del dataset:", df.shape)
display(df.head())


In [ ]:
# ============================================
# 4. Visualizaciones rápidas 
# ============================================

# Mapa de valores nulos
sns.heatmap(df.isnull(), cbar=False)
plt.title("Mapa de valores nulos")
plt.show()

# Boxplot solo para columnas numéricas con datos suficientes
for col in num_cols:
    if df[col].dropna().nunique() > 1:  # al menos dos valores distintos
        plt.figure(figsize=(6,3))
        sns.boxplot(x=df[col])
        plt.title(f"Outliers en {col}")
        plt.show()
    else:
        print(f"⚠️ Columna '{col}' omitida: no tiene suficientes datos numéricos válidos para graficar.")


In [ ]:
# Exportar el DataFrame limpio a un nuevo CSV
df.to_csv("dataset_limpio.csv", index=False, encoding="utf-8")
